In [1]:
"""
Имеются данные социальной игры в жанре match-3 на платформе Facebook.

Данные для тестового задания брать тут:
https://docs.google.com/spreadsheets/d/1Pmx-AWbYoCaee7FQZ06gkUbAzotgg2wLD7v2XfEXW4Y/edit?usp=sharing

На странице FirstEnter приложенной таблицы представлены данные о прохождении игроками туториала.
Туториал состоит из 8 шагов, выполняемых последовательно при первом входе в игру.
MaxAchievedStep: 0-8 - последний достигнутый игроком шаг туториала.

    0 - регистрация игрока
    1 - первый матч 3-х фишек
    2 - второй матч 3-х фишек
    3 - свободный ход (не ограничен туториалом)
    4 - окно награды за уровень с кнопкой “далее”
    5 - матч из 4-х фишек
    6 - матч бонуса (уничтожить линию)
    7 - свободный ход
    8 - конец уровня и конец туториала

На странице RetentionRate представлены данные о дате и времени входа игроков. 
Одинаковый UserId - один игрок. 
Данные об источнике инсталла можно получить из таблицы FirstEnter (столбец RefPlace). 
"""

In [94]:
#Задание:

import plotly.express as px
import numpy as np
import pandas as pd


data = pd.read_excel('Trainee Analyst Test DataSet.xlsx', ['FirstEnter', 'RetentionRate']) #считываем оба листа таблицы

#1. Построить график воронки туториала по шагам. Для построения можно использовать любой известный вам инструмент
        #Необходимо посчитать, сколько людей выполнило каждый из шагов и 
        #показать отношение количества людей, прошедших отдельные шаги туториала к количеству людей, стартовавших туториал.

#группируем игроков по шагам:
pivot = data['FirstEnter'].pivot_table(index ='Max AchievedStep', values = 'PlayerUid', aggfunc='count')  

#считаем общее число стартовавших туториал:
total_begin = int(pivot.sum())  

# добавляем новый столбец Total_Step = кол-во игроков, выполнивших данный этап
pivot['Total_Step'] = pivot['PlayerUid'][::-1].cumsum()[::-1]  

#добавляем новый столбец Rate = отношение количества людей, прошедших отдельные шаги туториала к количеству людей, стартовавших туториал
pivot['Rate'] = round(pivot['Total_Step']/total_begin,2)  

#добавляем новый столбец Step - расшифровка каждого шага
pivot['Step'] = ["0 - регистрация игрока", "1 - первый матч 3-х фишек", "2 - второй матч 3-х фишек", 
          "3 - свободный ход (не ограничен туториалом)", "4 - окно награды за уровень с кнопкой Далее",
         "5 - матч из 4-х фишек","6 - матч бонуса (уничтожить линию)","7 - свободный ход",
         "8 - конец уровня и конец туториала"]

#выводим саму воронку
fig = px.funnel(pivot, x='Total_Step', y='Step')
fig.show()


In [92]:
#2. Посчитать показатель Retention Day 1.
        #Retention Day X - это доля пользователей, которые вошли в приложение в конкретный день X после первого входа, 
        #т.е. Retention Day 1 - отношение количества людей, которые зашли на следующий день после первого визита 
        #к общему количеству совершивших первый вход.

data['RetentionRate']['Days']=data['RetentionRate']['DateTime'] - data['RetentionRate']['Registered']
pivot2 = data['RetentionRate'].pivot_table(columns = 'Days', index ='PlayerUid', values = 'DateTime', aggfunc='count')  #группируем 
pivot2.columns = ['days_0','days_1','days_2','days_3','days_4','days_5']
Retention_Day_1 = round(pivot2.days_1.count()/pivot2.days_0.count()*100, 2)
print('показатель Retention Day 1 =', Retention_Day_1, '%')

показатель Retention Day 1 = 8.56 %
